In [2]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.1 MB/s eta 0:00:00


In [4]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 9.8 MB/s eta 0:00:00


In [5]:
import pandas as pd
import re
import nltk
import json
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import emoji
import unicodedata

In [6]:
df = pd.read_csv('/content/drive/MyDrive/SMT 7/Pemrosesan Teks Teori/label_manual.csv')
df

,DATA,LABEL
0,mbappe mangan mbg,negative
1,cmn ide aja urusin aja tuh mbg cuan,negative
2,mbg hasil gaada jelas indikator output outcome...,negative
3,bilang tuh maksud hati nasi kuning or nasi udu...,negative
4,mbg bangsa bangun anakanak sehat,positive
...,...,...
245,masyarakat papua rasa manfaat program mbg kope...,positive
246,program mbg makan gratis peduli kerja cinta ta...,positive
247,arah presiden prabowo perintah atur khusus pro...,positive
248,salut guru kepala sekolah orangorang mbg tuh r...,positive


In [7]:
import re
import unicodedata
import emoji
import pandas as pd

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")

# Stemmer & Stopword base
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words("indonesian"))

# stopword tambahan sesuai dataset tidak formal Indonesia
stopwords_extra = {
    "nya", "sih", "saja", "aja", "kok", "pun", "lah", "dong", "deh",
    "banget", "bgt", "yg", "ya", "kan", "kah", "nah", "loh", "tuh", "nih",
    "gitu", "gini", "begini", "begitu", "oke", "ok",
    "yaa", "yaaah", "yah", "iyaa", "iya",
    "engga", "nggak", "gak", "ga", "amp", "gt",
    "bro", "sis", "min", "bang", "kak", "anda",
    "wkwk", "wkwkwk", "wk", "hehe", "haha", "hahaha",
    "dm", "pm", "rt", "btw", "jd", "kayak", "kayaknya"
}

stop_words.update(stopwords_extra)

bentuk_kata = re.compile(r"^[a-z]+$")

def reduce_repeated_chars(word):
    return re.sub(r"(.)\1{2,}", r"\1", word)

def remove_non_latin(text):
    return re.sub(r"[^\x00-\x7F]+", " ", text)

def preprocess(text):
    if pd.isna(text):
        return ""

    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = emoji.replace_emoji(text, replace=" ")
    text = re.sub(r"[@#]\w+", " ", text)
    text = remove_non_latin(text)
    text = re.sub(r"[^a-z\s]", " ", text)

    words = text.split()
    cleaned = []

    for w in words:
        w = reduce_repeated_chars(w)

        if not bentuk_kata.match(w):
            continue
        if w in stop_words:
            continue

        w = stemmer.stem(w)

        if w in stop_words:
            continue

        cleaned.append(w)

    return " ".join(cleaned)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
# Sesuaikan nama file kalau beda lokasi
df = pd.read_csv("/content/drive/MyDrive/SMT 7/Pemrosesan Teks Teori/label_manual.csv")

# bikin kolom teks bersih dari kolom DATA
df["bersih"] = df["DATA"].astype(str).apply(preprocess)

print(df[["DATA", "bersih", "LABEL"]].head())

                                                DATA  \
0                                  mbappe mangan mbg   
1                cmn ide aja urusin aja tuh mbg cuan   
2  mbg hasil gaada jelas indikator output outcome...   
3  bilang tuh maksud hati nasi kuning or nasi udu...   
4                   mbg bangsa bangun anakanak sehat   

                                              bersih     LABEL  
0                                  mbappe mangan mbg  negative  
1                            cmn ide urusin mbg cuan  negative  
2  mbg hasil gaada indikator output outcome nguku...  negative  
3  bilang maksud hati nasi kuning or nasi uduk et...  negative  
4                   mbg bangsa bangun anakanak sehat  positive  


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
)

X = df["bersih"]
y_raw = df["LABEL"]

# encode label string -> angka
le = LabelEncoder()
y = le.fit_transform(y_raw)
label_names = le.classes_

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# helper untuk plot confusion matrix
def plot_cm(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=label_names)
    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(ax=ax, cmap="Blues", xticks_rotation=45)
    plt.title(title)
    plt.tight_layout()
    plt.show()

### 1. BoW + Naive Bayes ###

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

print("\n===================================")
print("MODEL 1 : BoW (Bag-of-Words)")
print("===================================")

bow_scenarios = [
    {"name": "BoW_unigram_3000", "vect": {"max_features": 3000, "ngram_range": (1, 1)}, "clf": {"alpha": 1.0}},
    {"name": "BoW_unigram_5000", "vect": {"max_features": 5000, "ngram_range": (1, 1)}, "clf": {"alpha": 1.0}},
    {"name": "BoW_uni+bi_5000",  "vect": {"max_features": 5000, "ngram_range": (1, 2)}, "clf": {"alpha": 1.0}},
    {"name": "BoW_unigram_5000_alpha0.5", "vect": {"max_features": 5000, "ngram_range": (1, 1)}, "clf": {"alpha": 0.5}},
]

best_bow_name = ""
best_bow_acc = 0
best_bow_pred = None

for sc in bow_scenarios:
    print("\n----------------------")
    print(f"Running: {sc['name']}")
    print("----------------------")

    vect = CountVectorizer(**sc["vect"])
    X_train_bow = vect.fit_transform(X_train)
    X_test_bow = vect.transform(X_test)

    clf = MultinomialNB(**sc["clf"])
    clf.fit(X_train_bow, y_train)

    pred = clf.predict(X_test_bow)
    acc = accuracy_score(y_test, pred)
    print(f"Akurasi: {acc:.4f}")

    if acc > best_bow_acc:
        best_bow_acc = acc
        best_bow_name = sc["name"]
        best_bow_pred = pred

print("\n=== HASIL TERBAIK MODEL BoW ===")
print(f"Skenario terbaik : {best_bow_name}")
print(f"Akurasi terbaik  : {best_bow_acc:.4f}\n")
print("Classification Report (BoW BEST):")
print(classification_report(y_test, best_bow_pred, target_names=label_names))


MODEL 1 : BoW (Bag-of-Words)

----------------------
Running: BoW_unigram_3000
----------------------
Akurasi: 0.6000

----------------------
Running: BoW_unigram_5000
----------------------
Akurasi: 0.6000

----------------------
Running: BoW_uni+bi_5000
----------------------
Akurasi: 0.6000

----------------------
Running: BoW_unigram_5000_alpha0.5
----------------------
Akurasi: 0.5600

=== HASIL TERBAIK MODEL BoW ===
Skenario terbaik : BoW_unigram_3000
Akurasi terbaik  : 0.6000

Classification Report (BoW BEST):
              precision    recall  f1-score   support

    negative       0.65      0.55      0.59        20
     neutral       0.40      0.20      0.27        10
    positive       0.61      0.85      0.71        20

    accuracy                           0.60        50
   macro avg       0.55      0.53      0.52        50
weighted avg       0.58      0.60      0.57        50



2. TF-IDF + LinearSVC (SVM)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

print("\n===================================")
print("MODEL 2 : TF-IDF")
print("===================================")

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

tfidf_scenarios = [
    {"name": "C=0.1",            "params": {"C": 0.1}},
    {"name": "C=1",              "params": {"C": 1}},
    {"name": "C=10",             "params": {"C": 10}},
    {"name": "C=100",            "params": {"C": 100}},
    {"name": "C=1 + balanced",   "params": {"C": 1, "class_weight": "balanced"}},
]

best_tfidf_name = ""
best_tfidf_acc = 0
best_tfidf_pred = None

for sc in tfidf_scenarios:
    print("\n----------------------")
    print(f"Running: {sc['name']}")
    print("----------------------")

    model = LinearSVC(**sc["params"])
    model.fit(X_train_tfidf, y_train)

    pred = model.predict(X_test_tfidf)
    acc = accuracy_score(y_test, pred)
    print(f"Akurasi: {acc:.4f}")

    if acc > best_tfidf_acc:
        best_tfidf_acc = acc
        best_tfidf_name = sc["name"]
        best_tfidf_pred = pred

print("\n=== HASIL TERBAIK MODEL TF-IDF ===")
print(f"Skenario terbaik : {best_tfidf_name}")
print(f"Akurasi terbaik  : {best_tfidf_acc:.4f}\n")
print("Classification Report (TF-IDF BEST):")
print(classification_report(y_test, best_tfidf_pred, target_names=label_names))


MODEL 2 : TF-IDF

----------------------
Running: C=0.1
----------------------
Akurasi: 0.6600

----------------------
Running: C=1
----------------------
Akurasi: 0.6800

----------------------
Running: C=10
----------------------
Akurasi: 0.6600

----------------------
Running: C=100
----------------------
Akurasi: 0.6800

----------------------
Running: C=1 + balanced
----------------------
Akurasi: 0.6600

=== HASIL TERBAIK MODEL TF-IDF ===
Skenario terbaik : C=1
Akurasi terbaik  : 0.6800

Classification Report (TF-IDF BEST):
              precision    recall  f1-score   support

    negative       0.72      0.65      0.68        20
     neutral       0.62      0.50      0.56        10
    positive       0.67      0.80      0.73        20

    accuracy                           0.68        50
   macro avg       0.67      0.65      0.66        50
weighted avg       0.68      0.68      0.68        50



/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


3. Word2Vec + SVM

In [17]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 27.5 MB/s eta 0:00:00


In [19]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.svm import SVC

print("\n===================================")
print("MODEL 3 : Word2Vec")
print("===================================")

sentences_train = [doc.split() for doc in X_train]

w2v_scenarios = [
    {"name": "dim100_win5",  "params": {"vector_size": 100, "window": 5}},
    {"name": "dim200_win5",  "params": {"vector_size": 200, "window": 5}},
    {"name": "dim100_win3",  "params": {"vector_size": 100, "window": 3}},
]

def avg_w2v_vector(text, model, dim):
    tokens = text.split()
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(dim)
    return np.mean(vectors, axis=0)

best_w2v_name = ""
best_w2v_acc = 0
best_w2v_pred = None

for sc in w2v_scenarios:
    print("\n----------------------")
    print(f"Running: {sc['name']}")
    print("----------------------")

    # train Word2Vec untuk skenario ini
    w2v_model = Word2Vec(
        sentences=sentences_train,
        min_count=1,
        workers=4,
        **sc["params"]
    )

    dim = sc["params"]["vector_size"]

    X_train_w2v = np.array([avg_w2v_vector(doc, w2v_model, dim) for doc in X_train])
    X_test_w2v  = np.array([avg_w2v_vector(doc, w2v_model, dim) for doc in X_test])

    clf = SVC(kernel="linear")
    clf.fit(X_train_w2v, y_train)

    pred = clf.predict(X_test_w2v)
    acc = accuracy_score(y_test, pred)
    print(f"Akurasi: {acc:.4f}")

    if acc > best_w2v_acc:
        best_w2v_acc = acc
        best_w2v_name = sc["name"]
        best_w2v_pred = pred

print("\n=== HASIL TERBAIK MODEL Word2Vec ===")
print(f"Skenario terbaik : {best_w2v_name}")
print(f"Akurasi terbaik  : {best_w2v_acc:.4f}\n")
print("Classification Report (Word2Vec BEST):")
print(classification_report(y_test, best_w2v_pred, target_names=label_names))


MODEL 3 : Word2Vec

----------------------
Running: dim100_win5
----------------------
Akurasi: 0.4800

----------------------
Running: dim200_win5
----------------------
Akurasi: 0.4800

----------------------
Running: dim100_win3
----------------------
Akurasi: 0.4200

=== HASIL TERBAIK MODEL Word2Vec ===
Skenario terbaik : dim100_win5
Akurasi terbaik  : 0.4800

Classification Report (Word2Vec BEST):
              precision    recall  f1-score   support

    negative       0.50      0.75      0.60        20
     neutral       0.00      0.00      0.00        10
    positive       0.45      0.45      0.45        20

    accuracy                           0.48        50
   macro avg       0.32      0.40      0.35        50
weighted avg       0.38      0.48      0.42        50



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


4. LSTM

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

print("\n===================================")
print("MODEL 4 : LSTM")
print("===================================")

max_vocab = 10000
max_len = 30   # bisa disesuaikan

tokenizer = Tokenizer(num_words=max_vocab, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post", truncating="post")
X_test_pad  = pad_sequences(X_test_seq, maxlen=max_len, padding="post", truncating="post")

vocab_size = min(max_vocab, len(tokenizer.word_index) + 1)
num_classes = len(label_names)

lstm_scenarios = [
    {"name": "LSTM_64_3epoch",  "units": 64,  "epochs": 3},
    {"name": "LSTM_128_3epoch", "units": 128, "epochs": 3},
    {"name": "LSTM_128_5epoch", "units": 128, "epochs": 5},
]

best_lstm_name = ""
best_lstm_acc = 0
best_lstm_pred = None

for sc in lstm_scenarios:
    print("\n----------------------")
    print(f"Running: {sc['name']}")
    print("----------------------")

    model_lstm = Sequential()
    model_lstm.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_len))
    model_lstm.add(LSTM(sc["units"]))
    model_lstm.add(Dense(num_classes, activation="softmax"))

    model_lstm.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )

    model_lstm.fit(
        X_train_pad,
        y_train,
        epochs=sc["epochs"],
        batch_size=32,
        validation_split=0.1,
        verbose=1
    )

    loss, acc = model_lstm.evaluate(X_test_pad, y_test, verbose=0)
    print(f"Akurasi: {acc:.4f}")

    if acc > best_lstm_acc:
        best_lstm_acc = acc
        best_lstm_name = sc["name"]

        y_prob = model_lstm.predict(X_test_pad)
        best_lstm_pred = np.argmax(y_prob, axis=1)

print("\n=== HASIL TERBAIK MODEL LSTM ===")
print(f"Skenario terbaik : {best_lstm_name}")
print(f"Akurasi terbaik  : {best_lstm_acc:.4f}\n")
print("Classification Report (LSTM BEST):")
print(classification_report(y_test, best_lstm_pred, target_names=label_names))


MODEL 4 : LSTM

----------------------
Running: LSTM_64_3epoch
----------------------
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.3527 - loss: 1.0860 - val_accuracy: 0.5000 - val_loss: 1.1093
Epoch 2/3
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3678 - loss: 1.0527 - val_accuracy: 0.2500 - val_loss: 1.1645
Epoch 3/3
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4853 - loss: 1.0332 - val_accuracy: 0.2500 - val_loss: 1.1695
Akurasi: 0.4000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step

----------------------
Running: LSTM_128_3epoch
----------------------
Epoch 1/3
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - accuracy: 0.3786 - loss: 1.0832 - val_accuracy: 0.2500 - val_loss: 1.1362
Epoch 2/3
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.4405 - loss: 1.0455 - val_accuracy: 0.5000 - val_loss: 1.1391
Epoch 3/3
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.4644 - loss: 1.0349 - val_accuracy: 0.3000 - val_loss: 1.1437
Akurasi: 0.3800

----------------------
Running: LSTM_128_5epoch
----------------------
Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 189ms/step -

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


5. Naive Bayes (dengan fitur TF-IDF)

In [22]:
from sklearn.naive_bayes import MultinomialNB

print("\n===================================")
print("MODEL 5 : Naive Bayes (TF-IDF)")
print("===================================")

nb_scenarios = [
    {"name": "NB_alpha1.0",          "params": {"alpha": 1.0}},
    {"name": "NB_alpha0.5",          "params": {"alpha": 0.5}},
    {"name": "NB_alpha0.1",          "params": {"alpha": 0.1}},
    {"name": "NB_alpha1.0_noPrior",  "params": {"alpha": 1.0, "fit_prior": False}},
]

best_nb_name = ""
best_nb_acc = 0
best_nb_pred = None

for sc in nb_scenarios:
    print("\n----------------------")
    print(f"Running: {sc['name']}")
    print("----------------------")

    nb = MultinomialNB(**sc["params"])
    nb.fit(X_train_tfidf, y_train)

    pred = nb.predict(X_test_tfidf)
    acc = accuracy_score(y_test, pred)
    print(f"Akurasi: {acc:.4f}")

    if acc > best_nb_acc:
        best_nb_acc = acc
        best_nb_name = sc["name"]
        best_nb_pred = pred

print("\n=== HASIL TERBAIK MODEL Naive Bayes (TF-IDF) ===")
print(f"Skenario terbaik : {best_nb_name}")
print(f"Akurasi terbaik  : {best_nb_acc:.4f}\n")

print("Classification Report (Naive Bayes BEST):")
print(classification_report(y_test, best_nb_pred, target_names=label_names))


MODEL 5 : Naive Bayes (TF-IDF)

----------------------
Running: NB_alpha1.0
----------------------
Akurasi: 0.6000

----------------------
Running: NB_alpha0.5
----------------------
Akurasi: 0.6200

----------------------
Running: NB_alpha0.1
----------------------
Akurasi: 0.5800

----------------------
Running: NB_alpha1.0_noPrior
----------------------
Akurasi: 0.5800

=== HASIL TERBAIK MODEL Naive Bayes (TF-IDF) ===
Skenario terbaik : NB_alpha0.5
Akurasi terbaik  : 0.6200

Classification Report (Naive Bayes BEST):
              precision    recall  f1-score   support

    negative       0.72      0.65      0.68        20
     neutral       1.00      0.10      0.18        10
    positive       0.55      0.85      0.67        20

    accuracy                           0.62        50
   macro avg       0.76      0.53      0.51        50
weighted avg       0.71      0.62      0.58        50

